In [74]:
!python --version

Python 2.7.12


In [75]:
from __future__ import print_function
# 이걸 넣어주면 (권장사항) 비록 우리는 python 2 버전을 사용중이지만 
# print 뒤에 괄호 넣지 않으면 에러가 나게 된다.
# 즉 마치 python 3처럼 요구한다.

In [76]:

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [77]:
import os
import numpy as np
import pickle

In [78]:
batch_size = 32
num_classes = 10
epochs = 3
data_augmentation = True
num_predictions = 20 # 최후에 예측 테스트해 볼 테스트 데이터 개수
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# 데이터 준비하기

In [79]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [80]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


### 50,000개 데이터 너무 크니까 500개로 줄여주자 (내가 추가한 내용)

In [81]:
num_training = 500
mask = list(range(num_training))

x_train = x_train[mask] # (500, 32, 32, 3)
y_train = y_train[mask] # (500, 10)

# 모델 만들기

In [82]:
model= Sequential()

In [83]:
model.add(Conv2D(32, (3,3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))          

In [84]:
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [85]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [86]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_26 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_27 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 15, 15, 64)        18496     
__________

# 컴파일

In [87]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.rmsprop(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# 데이터 전처리

In [88]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [89]:
x_train /= 255
x_test /= 255

# 모델 학습시키기

Data augmentation 유무에 따라 약간 다르게 전개됨

In [90]:
if not data_augmentation:
    print("Not using data augmentation.")
    model.fit(x_train, y_train, 
              batch_size=batch_size, 
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0],
                        epochs=epochs,
                        validation_data=(x_test, y_test))    

Using real-time data augmentation
Epoch 1/3
500/500 [==============================] - 140s - loss: 1.9755 - acc: 0.2724 - val_loss: 1.8037 - val_acc: 0.3523

# 모델과 가중치 저장

### 모델 저장

In [91]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('saved trained model at %s' % model_path)

saved trained model at /Users/andymac/dev/tf_python2/code/keras/saved_models/keras_cifar10_trained_model.h5


### 예측 결과에 사용할 레이블명 로드

In [95]:
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# 테스트 데이터로 모델 평가하기 

In [96]:
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=batch_size),
                                    steps=x_test.shape[0] // batch_size)


In [97]:
print("Model accuracy = %.2f" % (evaluation[1]))

Model accuracy = 0.38


### 몇개 샘플에 대해 테스트해 보기

In [98]:
predict_gen = model.predict_generator(datagen.flow(x_test, y_test, batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

In [104]:
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    
    if actual_label == predicted_label:
        print("[Correct]   " + 'Actual label = %s vs. Predicted label = %s' % (actual_label, predicted_label))
    else:
        print("[Incorrect] " + 'Actual label = %s vs. Predicted label = %s' % (actual_label, predicted_label))
    
    if predict_index == num_predictions:
        break

[Incorrect] Actual label = cat vs. Predicted label = dog
[Incorrect] Actual label = ship vs. Predicted label = deer
[Incorrect] Actual label = ship vs. Predicted label = deer
[Incorrect] Actual label = airplane vs. Predicted label = horse
[Incorrect] Actual label = frog vs. Predicted label = dog
[Incorrect] Actual label = frog vs. Predicted label = airplane
[Incorrect] Actual label = automobile vs. Predicted label = truck
[Incorrect] Actual label = frog vs. Predicted label = deer
[Incorrect] Actual label = cat vs. Predicted label = horse
[Incorrect] Actual label = automobile vs. Predicted label = frog
[Incorrect] Actual label = airplane vs. Predicted label = automobile
[Incorrect] Actual label = truck vs. Predicted label = deer
[Incorrect] Actual label = dog vs. Predicted label = ship
[Correct]   Actual label = horse vs. Predicted label = horse
[Incorrect] Actual label = truck vs. Predicted label = ship
[Incorrect] Actual label = ship vs. Predicted label = airplane
[Incorrect] Actual l